# Assignment - Game: Ghosts

## Instructions

This is a **self-contained notebook** - everything you need is here!

### Quick Start
1. **Run all cells** up to Section 5 (this loads the game client)
2. **Implement your solver** in Section 5
3. **Update configuration** in Section 7
4. **Play the game** in Section 7

### What You Need To Do
- Focus ONLY on implementing `my_agent()` function (Section 5)
- You can create various players, you'll be able to select your preferred one
- Everything else is provided for you!

### About Ghosts
Ghosts is a 2-player bluffing/deduction board game where:
- 6x6 board with exits at the four corners
- Each player has 8 ghosts: 4 good (blue) and 4 evil (red)
- Ghosts are hidden - opponents can't see which type until captured
- Players arrange their ghosts at the start in two rows (columns 1-4 only)
- Goal: Either (1) capture all opponent's good ghosts, OR (2) get a good ghost to an exit on opponent's side
- Lose if: (1) all your good ghosts are captured, OR (2) you capture all opponent's evil ghosts
- Movement: One square orthogonally (up, down, left, right)
- Capture: Move onto opponent's ghost to capture it (reveals both ghosts' types)

---
## Section 1: Setup

**Run this cell (no changes needed)**

In [8]:
import requests
import json
import time
import random
from collections import deque
from typing import List, Optional, Tuple, Any, Dict
from copy import deepcopy

print("✅ Dependencies imported")

BASE_URL = 'https://ie-aireasoning-gr4r5bl6tq-ew.a.run.app'  # Your Cloud Run URL

AGENT_RANDOM_SEED = None  # Set to an int for reproducible setups/moves during testing

if AGENT_RANDOM_SEED is not None:
    random.seed(AGENT_RANDOM_SEED)

print("✅ Configuration loaded")


✅ Dependencies imported
✅ Configuration loaded


---
## Section 2: Game Client Library

**Run this cell (no changes needed)**

This defines the game client that handles all server communication.

In [9]:
class GameClient:
    def __init__(self, base_url: str, token: str, debug: bool = False):
        self.base_url = base_url.rstrip('/')
        self.token = token
        self.debug = debug

    def _make_request(self, endpoint: str, params: dict, max_retries: int = 10) -> dict:
        params['TOKEN'] = self.token
        url = f'{self.base_url}{endpoint}'

        for attempt in range(max_retries):
            try:
                if self.debug:
                    print(f"[DEBUG] Request: {endpoint}")
                    print(f"[DEBUG] Params: {params}")

                response = requests.get(url, params=params, timeout=30)

                if self.debug:
                    print(f"[DEBUG] Response [{response.status_code}]: {response.text[:200]}")

                if response.status_code == 200:
                    if response.text:
                        try:
                            return response.json()
                        except (json.JSONDecodeError, ValueError) as e:
                            if self.debug:
                                print(f"[DEBUG] Non-JSON response: {response.text[:100]}")
                            return {}
                    return {}
                else:
                    print(f"⚠️  HTTP {response.status_code}: {response.text[:200]}")

            except requests.exceptions.Timeout:
                print(f"⚠️  Request timeout (attempt {attempt + 1}/{max_retries})")
            except requests.exceptions.RequestException as e:
                print(f"⚠️  Request error: {e} (attempt {attempt + 1}/{max_retries})")
            except Exception as e:
                print(f"⚠️  Unexpected error: {type(e).__name__}: {e} (attempt {attempt + 1}/{max_retries})")

            if attempt < max_retries - 1:
                time.sleep(1)

        raise Exception(f"Failed to connect to {endpoint} after {max_retries} attempts")

    def create_match(self, game_type: str, num_games: int, multiplayer: bool = False) -> str:
        response = self._make_request('/new-match', {
            'game-type': game_type,
            'num-games': str(num_games),
            'multi-player': 'True' if multiplayer else 'False'
        })
        
        if 'match-id' not in response:
            print(f"❌ Server response missing 'match-id'. Response: {response}")
            raise KeyError(f"Server response missing 'match-id'. Got: {response}")
        
        return response['match-id']

    def join_match(self, match_id: str) -> dict:
        response = self._make_request('/join-match', {
            'match-id': match_id
        })
        return response

    def get_game_state(self, match_id: str, game_index: int) -> dict:
        return self._make_request('/game-state-in-match', {
            'match-id': match_id,
            'game-index': str(game_index)
        })

    def get_match_state(self, match_id: str) -> dict:
        return self._make_request('/match-state', {
            'match-id': match_id
        })

    def make_move(self, match_id: str, player: str, move: Any) -> bool:
        move_str = move if isinstance(move, str) else json.dumps(move)
        
        self._make_request('/make-move-in-match', {
            'match-id': match_id,
            'player': player,
            'move': move_str
        })
        return True

print("✅ GameClient loaded")


def play_game(
    solver,
    base_url: str,
    token: str,
    game_type: str,
    game_class,
    multiplayer: bool = False,
    match_id: Optional[str] = None,
    num_games: int = 1,
    debug: bool = False,
    verbose: bool = True
) -> Tuple:
    client = GameClient(base_url, token, debug=debug)

    if match_id is None:
        if verbose:
            print(f"🎮 Creating new match: {num_games} x {game_type}")
        match_id = client.create_match(game_type, num_games, multiplayer)
        if verbose:
            print(f"   Match ID: {match_id}")

    if verbose:
        print(f"🔗 Joining match {match_id}...")
    match = client.join_match(match_id)
    player = match['player']
    num_games = match.get('num-games', num_games)
    if verbose:
        print(f"   You are player: {player}")

    game_state = client.get_game_state(match_id, 0)
    if game_state['status'] == 'waiting':
        if verbose:
            print("⏳ Waiting for opponent to join...")
        while game_state['status'] == 'waiting':
            time.sleep(2)
            game_state = client.get_game_state(match_id, 0)

    all_results = []
    wins = 0
    losses = 0
    draws = 0

    while True:
        match_state = client.get_match_state(match_id)
        if match_state['status'] != 'in_progress':
            break
        game_num = match_state['current-game-index']

        if verbose:
            print(f"\n{'='*50}")
            print(f"🎮 GAME {game_num + 1}/{num_games}")
            print(f"{'='*50}\n")

        # Get initial game state and check player assignment
        game_state = client.get_game_state(match_id, game_num)
        
        # Update player sign if it changed (randomized per game)
        if 'my-player' in game_state and game_state['my-player']:
            new_player = game_state['my-player']
            if new_player != player and verbose and game_num > 0:
                print(f"ℹ️  Player assignment changed: You are now Player {new_player}\n")
            player = new_player
        
        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)

        move_count = 0
        # Handle setup phase
        while game_state['status'] == 'setup':
            game_state = client.get_game_state(match_id, game_num)
            player = game_state.get('my-player', player)
            if 'winner' in game_state or game_state['status'] != 'setup':
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            
            # Check if this player needs to complete setup
            if game.is_setup_phase() and not game.setup_complete():
                if verbose:
                    print(f"\n🎯 SETUP PHASE - Player {player}")
                    print(f"   Submitting your ghost setup...")

                try:
                    setup = solver(game)
                    
                    if verbose:
                        print(f"   ✅ Setup submitted!")

                    client.make_move(match_id, player, setup)

                except Exception as e:
                    print(f"❌ Error in solver during setup: {e}")
                    import traceback
                    traceback.print_exc()
                    all_results.append(('error', None))
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent to complete setup...")
                time.sleep(2)

        # Now in playing phase - continue until complete
        while game_state['status'] == 'playing':
            game_state = client.get_game_state(match_id, game_num)
            player = game_state.get('my-player', player)
            if 'winner' in game_state or game_state['status'] == 'complete':
                break

            game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
            
            if game.is_terminal():
                break

            if verbose:
                game.print_board()

            if game.current_player == player:
                if verbose:
                    print(f"🤔 Your turn (Player {player})...")

                try:
                    move = solver(game)

                    if verbose and move and isinstance(move, list) and len(move) == 3:
                        print(f"   Moving ghost {move[0]} to ({move[1]}, {move[2]})")

                    client.make_move(match_id, player, move)
                    move_count += 1

                except Exception as e:
                    print(f"❌ Error in solver: {e}")
                    import traceback
                    traceback.print_exc()
                    all_results.append(('error', None))
                    break
            else:
                if verbose:
                    print(f"⏳ Waiting for opponent (Player {game.current_player})...")
                time.sleep(2)

        # game is complete
        game_state = client.get_game_state(match_id, game_num)
        game = game_class(game_state['state'], game_state['status'], game_state['player'], player)
        
        if verbose:
            game.print_board()
            print("=" * 40)

        winner = game_state.get('winner', '-')
        if winner == '-':
            if verbose:
                print("🤝 Game ended in a DRAW!")
            result = 'draw'
            draws += 1
        elif winner == player:
            if verbose:
                print("🎉 You WON! Congratulations!")
            result = 'win'
            wins += 1
        else:
            if verbose:
                print("😞 You LOST. Better luck next time!")
            result = 'loss'
            losses += 1

        all_results.append((result, player, winner))

        if verbose and num_games > 1:
            print(f"\n📊 Current Record: {wins}W - {losses}L - {draws}D")
            print(f"   Games Remaining: {num_games - game_num - 1}\n")

    # Return results
    stats = {
        'wins': wins,
        'losses': losses,
        'draws': draws,
        'total_games': num_games,
        'win_rate': wins / num_games if num_games > 0 else 0,
        'player': player,
        'match_id': match_id
    }

    return stats, all_results

print("✅ play_game loaded")

✅ GameClient loaded
✅ play_game loaded


---
## Section 3: Game State Class & Helper Functions

**Run this cell (no changes needed)**

This defines the `GhostsGame` class with all helper methods you'll need.

In [10]:
class GhostsGame:
    """
    Represents Ghosts game state with helper methods.
    
    Key methods for your solver:
    - game.get_my_pieces()             # Your pieces (with hidden types)
    - game.get_opponent_pieces()       # Opponent pieces (types hidden unless captured)
    - game.get_revealed_pieces()       # Opponent pieces you've captured (types revealed)
    - game.get_valid_moves()           # All valid moves for your pieces
    - game.is_exit(row, col)           # Check if position is a winning exit
    - game.simulate_move(move)         # Simulate move for search
    - game.is_terminal()               # Check if game over
    - game.is_setup_phase()            # Check if in setup phase
    - game.print_board()               # Debug visualization
    """

    def __init__(self, state: str, status: str, current_player: str, my_player: str):
        self.state_str = state
        self.status = status
        self.current_player = current_player
        self.my_player = my_player
        self._state = None
        self._valid_moves = None

    @property
    def state(self) -> Dict:
        """Get state dictionary."""
        if self._state is None:
            self._state = json.loads(self.state_str)
        return self._state

    def is_setup_phase(self) -> bool:
        """Check if game is in setup phase."""
        return self.status == 'setup' or self.state.get('phase') == 'setup'

    def setup_complete(self) -> bool:
        """Check if this player has completed setup."""
        if not self.is_setup_phase():
            return True
        return self.state.get('setup_complete', {}).get(self.my_player, False)

    def get_my_pieces(self) -> List[Dict]:
        """Get your pieces (you know their types)."""
        if self.my_player not in self.state.get('pieces', {}):
            return []
        return [p for p in self.state['pieces'][self.my_player] if not p.get('captured', False)]

    def get_opponent_pieces(self, include_captured: bool = False) -> List[Dict]:
        """Get opponent pieces (types hidden unless revealed by capture)."""
        opponent = '2' if self.my_player == '1' else '1'
        if opponent not in self.state.get('pieces', {}):
            return []
        pieces = self.state['pieces'][opponent]
        
        if include_captured:
            return pieces
        else:
            return [p for p in pieces if not p.get('captured', False)]

    def get_revealed_pieces(self) -> List[Dict]:
        """Get opponent pieces whose types have been revealed to you."""
        opponent = '2' if self.my_player == '1' else '1'
        revealed_ids = self.state.get('revealed', {}).get(opponent, [])
        
        revealed_pieces = []
        for piece in self.state.get('pieces', {}).get(opponent, []):
            if piece.get('id') in revealed_ids:
                revealed_pieces.append(piece)
        return revealed_pieces

    def is_terminal(self) -> bool:
        """Check if game is over."""
        return self.status == 'complete'

    def is_waiting(self) -> bool:
        """Check if waiting for opponent."""
        return self.status == 'waiting'

    def get_opponent(self, player: str) -> str:
        """Get opponent's identifier."""
        return '2' if player == '1' else '1'

    def is_exit(self, row: int, col: int) -> bool:
        """Check if position is a winning exit for your good ghosts."""
        # Player 1 wins at bottom exits (row 5, cols 0 and 5)
        # Player 2 wins at top exits (row 0, cols 0 and 5)
        if self.my_player == '1':
            return row == 5 and col in [0, 5]
        else:
            return row == 0 and col in [0, 5]

    def get_piece_at(self, row: int, col: int) -> Optional[Tuple[str, Dict]]:
        """Get piece at position. Returns (player, piece) or None."""
        for player in ['1', '2']:
            if player not in self.state.get('pieces', {}):
                continue
            for piece in self.state['pieces'][player]:
                if not piece.get('captured', False) and piece.get('row') == row and piece.get('col') == col:
                    return player, piece
        return None

    def get_valid_moves(self, player: Optional[str] = None) -> List[List[int]]:
        """Get all valid moves for player. Returns list of [piece_id, row, col]."""
        if player is None:
            player = self.my_player

        # No moves during setup
        if self.is_setup_phase():
            return []

        if self._valid_moves is None or player != self.current_player:
            moves = []
            for piece in self.state.get('pieces', {}).get(player, []):
                if piece.get('captured', False):
                    continue

                row, col = piece.get('row'), piece.get('col')
                piece_id = piece.get('id')

                # Try all four directions
                for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    new_row = row + dr
                    new_col = col + dc

                    # Check bounds
                    if 0 <= new_row < 6 and 0 <= new_col < 6:
                        # Check if square is occupied
                        occupant = self.get_piece_at(new_row, new_col)

                        # Exits at corners - can only move there if it's a winning move
                        is_corner = (new_row == 0 or new_row == 5) and (new_col == 0 or new_col == 5)
                        if is_corner:
                            # Only allow if it's a winning exit for a good ghost
                            if piece.get('type') == 'good' and self.is_exit(new_row, new_col):
                                moves.append([piece_id, new_row, new_col])
                            # Otherwise, corners are blocked (can't stop there)
                        else:
                            # Regular square: can move to empty square or capture opponent
                            if occupant is None or occupant[0] != player:
                                moves.append([piece_id, new_row, new_col])

            self._valid_moves = moves
        return self._valid_moves

    def simulate_move(self, move: List[int]) -> Dict:
        """
        Simulate a move and return new state.
        Does NOT contact server or modify original state.
        """
        new_state = deepcopy(self.state)
        player = self.current_player
        opponent = self.get_opponent(player)

        piece_id, new_row, new_col = move[0], move[1], move[2]

        # Find and move the piece
        for piece in new_state['pieces'][player]:
            if piece['id'] == piece_id:
                # Check if capturing
                for opp_piece in new_state['pieces'][opponent]:
                    if not opp_piece['captured'] and opp_piece['row'] == new_row and opp_piece['col'] == new_col:
                        opp_piece['captured'] = True
                        # Reveal BOTH pieces: captured piece AND capturing piece
                        if opp_piece['id'] not in new_state['revealed'][opponent]:
                            new_state['revealed'][opponent].append(opp_piece['id'])
                        if piece_id not in new_state['revealed'][player]:
                            new_state['revealed'][player].append(piece_id)
                        break

                # Move piece
                piece['row'] = new_row
                piece['col'] = new_col
                break

        return new_state

    def count_pieces(self, player: str, piece_type: str, state: Optional[Dict] = None) -> int:
        """Count remaining pieces of a type for a player."""
        if state is None:
            state = self.state

        count = 0
        for piece in state.get('pieces', {}).get(player, []):
            if not piece.get('captured', False) and piece.get('type') == piece_type:
                count += 1
        return count

    def print_board(self):
        """Print board visualization (works for both setup and playing phases)."""
        state = self.state
        opponent = self.get_opponent(self.my_player)

        # Header
        print("\n" + "=" * 50)
        if self.is_setup_phase():
            print(f"SETUP PHASE - Player {self.my_player}")
            print(f"Setup complete: {self.setup_complete()}")
        else:
            print(f"Player {self.my_player} (You) - Good: {self.count_pieces(self.my_player, 'good')}, Evil: {self.count_pieces(self.my_player, 'evil')}")
            print(f"Player {opponent} (Opp) - Good: {self.count_pieces(opponent, 'good')}, Evil: {self.count_pieces(opponent, 'evil')}")
        print("=" * 50)
        
        # Column headers
        print("  0   1   2   3   4   5")
        print("┌" + "───┬" * 5 + "───┐")

        # Board rows
        for r in range(6):
            row_str = ""
            for c in range(6):
                cell = "   "
                
                # Check for pieces from both players
                for player in ['1', '2']:
                    if player not in state.get('pieces', {}):
                        continue
                    for piece in state['pieces'][player]:
                        if not piece.get('captured', False) and piece.get('row') == r and piece.get('col') == c:
                            if player == self.my_player:
                                # Show your pieces with type
                                symbol = 'G' if piece.get('type') == 'good' else 'E'
                                cell = f" {symbol} "
                            else:
                                # Show opponent pieces (unknown unless revealed)
                                if piece.get('id') in state.get('revealed', {}).get(player, []):
                                    symbol = 'g' if piece.get('type') == 'good' else 'e'
                                    cell = f" {symbol} "
                                else:
                                    cell = " ? "
                            break
                
                # Mark exits at corners
                if (r == 0 or r == 5) and (c == 0 or c == 5):
                    if cell == "   ":
                        cell = " X "

                row_str += "│" + cell
            row_str += "│ " + str(r)
            print(row_str)
            
            if r < 5:
                print("├" + "───┼" * 5 + "───┤")

        print("└" + "───┴" * 5 + "───┘")
        
        # Legend
        if self.is_setup_phase():
            print("\nLegend: G=good ghost, E=evil ghost, X=exit")
            if self.my_player == '1':
                print("Your setup area: Rows 0-1, Columns 1-4")
            else:
                print("Your setup area: Rows 4-5, Columns 1-4")
            print("Note: Exits (X) can only be entered by good ghosts to win!")
        else:
            print("\nLegend: G=your good, E=your evil, ?=unknown opponent")
            print("        g=revealed good, e=revealed evil, X=exit")
        print()


# Setup helper functions
def get_default_setup(player: str) -> List[Dict]:
    """
    Get default piece arrangement for a player.
    
    Args:
        player: '1' or '2'
    
    Returns:
        List of 8 pieces with 'row', 'col', 'type' (columns 1-4 only)
    """
    if player == '1':
        # Player 1 default setup (rows 0-1, cols 1-4)
        return [
            {'row': 0, 'col': 1, 'type': 'good'},
            {'row': 0, 'col': 2, 'type': 'good'},
            {'row': 0, 'col': 3, 'type': 'evil'},
            {'row': 0, 'col': 4, 'type': 'evil'},
            {'row': 1, 'col': 1, 'type': 'good'},
            {'row': 1, 'col': 2, 'type': 'good'},
            {'row': 1, 'col': 3, 'type': 'evil'},
            {'row': 1, 'col': 4, 'type': 'evil'},
        ]
    else:  # player == '2'
        # Player 2 default setup (rows 4-5, cols 1-4)
        return [
            {'row': 4, 'col': 1, 'type': 'good'},
            {'row': 4, 'col': 2, 'type': 'good'},
            {'row': 4, 'col': 3, 'type': 'evil'},
            {'row': 4, 'col': 4, 'type': 'evil'},
            {'row': 5, 'col': 1, 'type': 'good'},
            {'row': 5, 'col': 2, 'type': 'good'},
            {'row': 5, 'col': 3, 'type': 'evil'},
            {'row': 5, 'col': 4, 'type': 'evil'},
        ]


def create_custom_setup(player: str, good_positions: List[Tuple[int, int]],
                       evil_positions: List[Tuple[int, int]]) -> List[Dict]:
    """
    Create a custom piece arrangement.

    Args:
        player: '1' or '2'
        good_positions: List of (row, col) tuples for good ghosts (must be 4, cols 1-4)
        evil_positions: List of (row, col) tuples for evil ghosts (must be 4, cols 1-4)

    Returns:
        List of 8 pieces with 'row', 'col', 'type'

    Example:
        setup = create_custom_setup('1',
                                   [(0, 1), (0, 2), (1, 1), (1, 2)],  # good ghosts
                                   [(0, 3), (0, 4), (1, 3), (1, 4)])  # evil ghosts
    """
    if len(good_positions) != 4 or len(evil_positions) != 4:
        raise ValueError("Must have exactly 4 good and 4 evil positions")

    # Validate setup area
    valid_rows = [0, 1] if player == '1' else [4, 5]
    valid_cols = [1, 2, 3, 4]
    all_positions = good_positions + evil_positions

    for row, col in all_positions:
        if row not in valid_rows:
            raise ValueError(f"Invalid row {row} for player {player}. Must be {valid_rows}")
        if col not in valid_cols:
            raise ValueError(f"Invalid column {col}. Must be in {valid_cols}")

    # Check for duplicate positions
    if len(set(all_positions)) != 8:
        raise ValueError("Positions must be unique")

    setup = []
    for row, col in good_positions:
        setup.append({'row': row, 'col': col, 'type': 'good'})
    for row, col in evil_positions:
        setup.append({'row': row, 'col': col, 'type': 'evil'})

    return setup


def manual_setup(player: str) -> List[Dict]:
    """
    Interactive manual setup - place your ghosts!
    
    Args:
        player: '1' or '2'
    
    Returns:
        List of 8 pieces with 'row', 'col', 'type'
    """
    print(f"\n🎮 GHOST SETUP for Player {player}")
    print("=" * 50)
    
    if player == '1':
        print("Your setup area: Rows 0-1, Columns 1-4")
        valid_rows = [0, 1]
    else:
        print("Your setup area: Rows 4-5, Columns 1-4")
        valid_rows = [4, 5]
    
    valid_cols = [1, 2, 3, 4]
    
    print("\nYou need to place:")
    print("  - 4 GOOD ghosts (these win by reaching opponent's exits)")
    print("  - 4 EVIL ghosts (if opponent captures all of these, you win!)")
    print("\nEnter positions as: row,col type (e.g., '0,1 good' or '1,3 evil')")
    print("Type 'default' to use default setup")
    print()
    
    setup = []
    good_count = 0
    evil_count = 0
    used_positions = set()
    
    while len(setup) < 8:
        remaining_good = 4 - good_count
        remaining_evil = 4 - evil_count
        
        print(f"\nPiece {len(setup) + 1}/8 - Need {remaining_good} good, {remaining_evil} evil")
        
        try:
            inp = input(f"Enter position and type (e.g., '{valid_rows[0]},1 good'): ").strip().lower()
            
            if inp == 'default':
                return get_default_setup(player)
            
            parts = inp.split()
            if len(parts) != 2:
                print("❌ Invalid format! Use: row,col type")
                continue
            
            pos_part, type_part = parts
            row, col = map(int, pos_part.split(','))
            
            if row not in valid_rows:
                print(f"❌ Invalid row! Must be {' or '.join(map(str, valid_rows))}")
                continue
            
            if col not in valid_cols:
                print(f"❌ Invalid column! Must be 1, 2, 3, or 4 (not 0 or 5)")
                continue
            
            if (row, col) in used_positions:
                print("❌ Position already used!")
                continue
            
            if type_part not in ['good', 'evil']:
                print("❌ Type must be 'good' or 'evil'")
                continue
            
            if type_part == 'good' and good_count >= 4:
                print("❌ Already have 4 good ghosts!")
                continue
            
            if type_part == 'evil' and evil_count >= 4:
                print("❌ Already have 4 evil ghosts!")
                continue
            
            setup.append({'row': row, 'col': col, 'type': type_part})
            used_positions.add((row, col))
            
            if type_part == 'good':
                good_count += 1
            else:
                evil_count += 1
            
            print(f"✅ Placed {type_part} ghost at ({row}, {col})")
            
        except (ValueError, IndexError):
            print("❌ Invalid input! Use format: row,col type")
        except KeyboardInterrupt:
            print("\n\n Using default setup...")
            return get_default_setup(player)
    
    print("\n✅ Setup complete!")
    return setup

print("✅ GhostsGame class and setup helpers loaded")

✅ GhostsGame class and setup helpers loaded


---
## Section 4: Setup & Play Functions

**Run this cell (no changes needed)**

These functions handle the setup phase and gameplay.

In [11]:
def manual_player_solver(game: GhostsGame) -> List:
    """
    Interactive manual player - YOU choose everything!
    Handles both setup and playing phases.
    """
    # SETUP PHASE
    if game.is_setup_phase() and not game.setup_complete():
        game.print_board()
        return manual_setup(game.my_player)
    
    # PLAYING PHASE
    game.print_board()
    
    moves = game.get_valid_moves()
    
    if not moves:
        print("No valid moves available!")
        return None
    
    print(f"\n🎮 YOUR TURN (Player {game.my_player})!")
    print("\nYour pieces and valid moves:")
    
    # Group moves by piece
    piece_moves = {}
    for move in moves:
        piece_id = move[0]
        if piece_id not in piece_moves:
            piece_moves[piece_id] = []
        piece_moves[piece_id].append(move)
    
    # Display moves
    move_list = []
    for piece_id, pmoves in piece_moves.items():
        piece = None
        for p in game.get_my_pieces():
            if p['id'] == piece_id:
                piece = p
                break
        
        if piece:
            print(f"\n  Ghost {piece_id} ({piece['type']}) at ({piece['row']}, {piece['col']}):")
            for move in pmoves:
                idx = len(move_list)
                move_list.append(move)
                
                # Check if capturing
                occupant = game.get_piece_at(move[1], move[2])
                if occupant:
                    print(f"    {idx}: Move to ({move[1]}, {move[2]}) [CAPTURE opponent ghost]")
                else:
                    print(f"    {idx}: Move to ({move[1]}, {move[2]})")
    
    while True:
        try:
            choice = input("\nEnter move number (or 'q' to quit): ").strip()
            
            if choice.lower() == 'q':
                raise KeyboardInterrupt()
            
            idx = int(choice)
            if 0 <= idx < len(move_list):
                return move_list[idx]
            else:
                print(f"❌ Invalid index! Choose 0-{len(move_list)-1}")
        
        except ValueError:
            print("❌ Invalid input! Enter a number.")
        except KeyboardInterrupt:
            print("\n👋 Thanks for playing!")
            raise

print("✅ Setup and play functions loaded")

✅ Setup and play functions loaded


---
## Section 5: YOUR SOLVER IMPLEMENTATION

**⭐ THIS IS WHERE YOU WRITE YOUR CODE! ⭐**

Implement your AI algorithm here. You can use:
- Minimax with expectimax (handling hidden information)
- Alpha-beta pruning
- Probabilistic reasoning about opponent's piece types
- Custom heuristics

### Available Methods

```python
game.get_my_pieces()                 # Your pieces (you know types)
game.get_opponent_pieces()           # Opponent pieces (types hidden)
game.get_revealed_pieces()           # Opponent pieces you've captured
game.get_valid_moves()               # All valid moves
game.is_exit(row, col)               # Check if winning exit
game.get_piece_at(row, col)          # Get piece at position
game.simulate_move(move)             # Simulate move
game.count_pieces(player, type)      # Count pieces
game.print_board()                   # Print board
```

### Move Format
- Move format: `[piece_id, row, col]`

### Strategy Tips
1. Try to deduce opponent ghost types from their movement patterns
2. Protect your good ghosts while advancing them to exits
3. Use evil ghosts as decoys or to capture opponent pieces
4. Balance aggression (capturing) vs. advancement (reaching exits)
5. Be careful! If you capture all opponent's evil ghosts, YOU LOSE!

In [12]:
def strategic_setup(player: str) -> List[Dict]:
    """
    Create a varied but sensible ghost setup for the given player.

    Args:
        player: '1' or '2' indicating which side to place.

    Returns:
        List of 8 piece dictionaries ready for submission.
    """
    if player == '1':
        patterns = [
            ([(1, 1), (1, 4), (0, 2), (0, 3)], [(1, 2), (0, 1), (0, 4), (1, 3)]),
            ([(1, 2), (1, 3), (0, 1), (0, 4)], [(1, 1), (1, 4), (0, 2), (0, 3)]),
            ([(1, 1), (1, 2), (0, 3), (0, 4)], [(1, 3), (1, 4), (0, 1), (0, 2)]),
            ([(1, 2), (1, 4), (0, 2), (0, 4)], [(1, 1), (1, 3), (0, 1), (0, 3)]),
        ]
    else:
        patterns = [
            ([(4, 1), (4, 4), (5, 2), (5, 3)], [(4, 2), (5, 1), (5, 4), (4, 3)]),
            ([(4, 2), (4, 3), (5, 1), (5, 4)], [(4, 1), (4, 4), (5, 2), (5, 3)]),
            ([(4, 1), (4, 2), (5, 3), (5, 4)], [(4, 3), (4, 4), (5, 1), (5, 2)]),
            ([(4, 2), (4, 4), (5, 2), (5, 4)], [(4, 1), (4, 3), (5, 1), (5, 3)]),
        ]

    good, evil = random.choice(patterns)
    return create_custom_setup(player, good, evil)


def manhattan_distance(r1: int, c1: int, r2: int, c2: int) -> int:
    """Return Manhattan distance between two cells."""
    return abs(r1 - r2) + abs(c1 - c2)


def is_exit_for_player(player: str, row: int, col: int) -> bool:
    """Check whether a coordinate is a valid exit for the given player."""
    exits = [(5, 0), (5, 5)] if player == '1' else [(0, 0), (0, 5)]
    return (row, col) in exits


def distance_to_exit(player: str, row: int, col: int) -> int:
    """
    Compute the minimum Manhattan distance from (row, col) to the player's exits.

    Args:
        player: Player id ('1' or '2').
        row: Current row of the ghost.
        col: Current column of the ghost.

    Returns:
        Manhattan distance to the nearest winning exit.
    """
    exits = [(5, 0), (5, 5)] if player == '1' else [(0, 0), (0, 5)]
    return min(manhattan_distance(row, col, er, ec) for er, ec in exits)


def count_pieces_in_state(state: Dict, player: str, piece_type: str) -> int:
    """Count non-captured pieces of a specific type for a player in the given state."""
    count = 0
    for piece in state.get('pieces', {}).get(player, []):
        if not piece.get('captured') and piece.get('type') == piece_type:
            count += 1
    return count


def get_opponent_id(player: str) -> str:
    """Return the opponent id for the given player."""
    return '2' if player == '1' else '1'


def build_position_map(state: Dict) -> Dict[Tuple[int, int], Tuple[str, Dict]]:
    pos: Dict[Tuple[int, int], Tuple[str, Dict]] = {}
    for player, pieces in state.get('pieces', {}).items():
        for piece in pieces:
            if piece.get('captured'):
                continue
            pos[(piece.get('row'), piece.get('col'))] = (player, piece)
    return pos


def unknown_type_counts(state: Dict, player: str) -> Tuple[int, int]:
    revealed = set(state.get('revealed', {}).get(player, []))
    unknown_good = 0
    unknown_evil = 0
    for piece in state.get('pieces', {}).get(player, []):
        if piece.get('captured'):
            continue
        if piece.get('id') in revealed:
            continue
        if piece.get('type') == 'good':
            unknown_good += 1
        elif piece.get('type') == 'evil':
            unknown_evil += 1
    return unknown_good, unknown_evil


def piece_type_probabilities(state: Dict, owner: str, piece: Dict) -> Tuple[float, float]:
    revealed = state.get('revealed', {}).get(owner, [])
    if piece.get('id') in revealed:
        if piece.get('type') == 'good':
            return (1.0, 0.0)
        return (0.0, 1.0)

    unknown_good, unknown_evil = unknown_type_counts(state, owner)
    total_unknown = unknown_good + unknown_evil
    if total_unknown == 0:
        prob_good = 0.5
        prob_evil = 0.5
    else:
        prob_good = unknown_good / total_unknown
        prob_evil = unknown_evil / total_unknown

    dist_to_exit_val = min(distance_to_exit(owner, piece.get('row'), piece.get('col')), 6)
    proximity_bias = max(0.0, (6 - dist_to_exit_val) * 0.03)
    home_rows = [0, 1] if owner == '1' else [4, 5]
    if piece.get('row') in home_rows:
        proximity_bias -= 0.05

    prob_good = max(0.0, prob_good + proximity_bias)
    prob_evil = max(0.0, prob_evil - proximity_bias)
    total = prob_good + prob_evil
    if total == 0:
        return (0.5, 0.5)
    return (prob_good / total, prob_evil / total)


def bfs_exit_distance(state: Dict, player: str, piece: Dict) -> Optional[int]:
    if piece.get('captured') or piece.get('type') != 'good':
        return None

    exits = [(5, 0), (5, 5)] if player == '1' else [(0, 0), (0, 5)]
    start = (piece.get('row'), piece.get('col'))
    pos_map = build_position_map(state)
    queue = deque([(start, 0)])
    seen = {start}

    while queue:
        (r, c), dist = queue.popleft()
        if (r, c) in exits:
            return dist

        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = r + dr, c + dc
            if nr < 0 or nr >= 6 or nc < 0 or nc >= 6:
                continue
            if (nr, nc) in seen:
                continue
            is_corner = (nr in [0, 5]) and (nc in [0, 5])
            if is_corner and (nr, nc) not in exits:
                continue
            occupant = pos_map.get((nr, nc))
            if occupant and occupant[0] == player:
                continue
            seen.add((nr, nc))
            queue.append(((nr, nc), dist + 1))

    return None


def state_signature(state: Dict) -> tuple:
    """Create a hashable signature for transposition caching."""
    pieces_sig = []
    for pl in ['1', '2']:
        pieces = [
            (p.get('id'), p.get('row'), p.get('col'), p.get('type'), bool(p.get('captured')))
            for p in state.get('pieces', {}).get(pl, [])
        ]
        pieces_sig.append(tuple(sorted(pieces)))
    revealed_sig = tuple((pl, tuple(sorted(state.get('revealed', {}).get(pl, [])))) for pl in ['1', '2'])
    return (tuple(pieces_sig), revealed_sig)


def detect_winner(state: Dict) -> Optional[str]:
    """
    Determine winner from a state snapshot based on capture/exit conditions.

    Returns:
        '1', '2', or None if game is not terminal.
    """
    p1_good = count_pieces_in_state(state, '1', 'good')
    p2_good = count_pieces_in_state(state, '2', 'good')
    p1_evil = count_pieces_in_state(state, '1', 'evil')
    p2_evil = count_pieces_in_state(state, '2', 'evil')

    # Exit victories
    for piece in state.get('pieces', {}).get('1', []):
        if not piece.get('captured') and piece.get('type') == 'good' and is_exit_for_player('1', piece['row'], piece['col']):
            return '1'
    for piece in state.get('pieces', {}).get('2', []):
        if not piece.get('captured') and piece.get('type') == 'good' and is_exit_for_player('2', piece['row'], piece['col']):
            return '2'

    # All good captured
    if p1_good == 0:
        return '2'
    if p2_good == 0:
        return '1'

    # Capturing all opponent evil ghosts loses you the game
    if p1_evil == 0 and p2_evil > 0:
        return '2'
    if p2_evil == 0 and p1_evil > 0:
        return '1'

    return None


def evaluate_state(
    game: GhostsGame,
    state: Optional[Dict] = None,
    perspective: Optional[str] = None
) -> float:
    """
    Heuristic evaluation of a GhostsGame state with lightweight terminal checks.

    Args:
        game: GhostsGame with helper utilities.
        state: State dict to score; defaults to the game's current state.
        perspective: Player id to favor; defaults to game.my_player.

    Returns:
        Score where higher is better for `perspective`.
    """
    if state is None:
        state = game.state

    me = perspective if perspective is not None else game.my_player
    opp = get_opponent_id(me)

    winner = detect_winner(state)
    if winner == me:
        return 1e6
    if winner == opp:
        return -1e6

    pieces = state.get('pieces', {})
    my_pieces = pieces.get(me, [])
    opp_pieces = pieces.get(opp, [])

    my_good = count_pieces_in_state(state, me, "good")
    my_evil = count_pieces_in_state(state, me, "evil")
    opp_good = count_pieces_in_state(state, opp, "good")
    opp_evil = count_pieces_in_state(state, opp, "evil")

    score = 14 * (my_good - opp_good)
    score += 4 * (my_evil - opp_evil)

    my_mobility = len(get_valid_moves_for_state(state, me))
    opp_mobility = len(get_valid_moves_for_state(state, opp))
    score += 0.9 * (my_mobility - opp_mobility)

    pos_map = build_position_map(state)
    exposure_penalty = 0.0
    my_paths: List[int] = []
    opp_paths: List[int] = []

    for piece in my_pieces:
        if piece.get('captured') or piece.get('type') != 'good':
            continue
        dist = distance_to_exit(me, piece['row'], piece['col'])
        path_len = bfs_exit_distance(state, me, piece)
        if path_len is not None:
            my_paths.append(path_len)
            score += max(0, 8 - path_len) * 3.5
        score += max(0, 6 - dist) * 1.2

    for piece in opp_pieces:
        if piece.get('captured'):
            continue
        prob_good, _ = piece_type_probabilities(state, opp, piece)
        dist = distance_to_exit(opp, piece['row'], piece['col'])
        path_len = bfs_exit_distance(state, opp, piece) if piece.get('type') == 'good' else None
        opp_paths.append(path_len if path_len is not None else dist)
        threat_range = path_len if path_len is not None else dist + 1
        score -= prob_good * max(0, 7 - threat_range) * 3.5

    for piece in my_pieces:
        if piece.get('captured') or piece.get('type') != 'good':
            continue
        r, c = piece['row'], piece['col']
        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            occ = pos_map.get((r + dr, c + dc))
            if occ and occ[0] == opp:
                prob_good, _ = piece_type_probabilities(state, opp, occ[1])
                exposure_penalty += 0.8 + prob_good
    score -= exposure_penalty * 2.0

    if my_paths:
        score -= 1.2 * (sum(my_paths) / len(my_paths))
    if opp_paths:
        score += 1.2 * (sum(opp_paths) / len(opp_paths))

    vision_advantage = len(state.get('revealed', {}).get(opp, [])) - len(state.get('revealed', {}).get(me, []))
    score += 0.6 * vision_advantage

    return score


def apply_move_to_state(
    state: Dict,
    player: str,
    move: List[int],
    override_capture_type: Optional[str] = None
) -> Dict:
    """
    Purely apply a move to a copied state and return the new state.

    Args:
        state: Current state snapshot.
        player: Player making the move.
        move: [piece_id, row, col].
        override_capture_type: If set, force the captured piece type to this value
            before marking captured (useful for probabilistic branching).
    """
    new_state = deepcopy(state)
    opponent = get_opponent_id(player)

    # Ensure revealed lists exist
    new_state.setdefault('revealed', {})
    new_state['revealed'].setdefault(player, [])
    new_state['revealed'].setdefault(opponent, [])

    piece_id, new_row, new_col = move

    # Capture if needed
    for opp_piece in new_state.get('pieces', {}).get(opponent, []):
        if not opp_piece.get('captured') and opp_piece.get('row') == new_row and opp_piece.get('col') == new_col:
            if override_capture_type is not None:
                opp_piece['type'] = override_capture_type
            opp_piece['captured'] = True
            if opp_piece['id'] not in new_state['revealed'][opponent]:
                new_state['revealed'][opponent].append(opp_piece['id'])
            break

    # Move our piece (and reveal it)
    for piece in new_state.get('pieces', {}).get(player, []):
        if piece['id'] == piece_id:
            piece['row'] = new_row
            piece['col'] = new_col
            if piece_id not in new_state['revealed'][player]:
                new_state['revealed'][player].append(piece_id)
            break

    return new_state


def get_valid_moves_for_state(state: Dict, player: str) -> List[List[int]]:
    """Generate legal moves for a player from a raw state snapshot."""
    moves: List[List[int]] = []
    pieces = state.get('pieces', {}).get(player, [])

    for piece in pieces:
        if piece.get('captured'):
            continue

        row, col = piece['row'], piece['col']
        pid = piece['id']

        for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nr, nc = row + dr, col + dc
            if nr < 0 or nr >= 6 or nc < 0 or nc >= 6:
                continue

            is_corner = (nr in [0, 5]) and (nc in [0, 5])
            occupant = None
            for opp in ['1', '2']:
                for opp_piece in state.get('pieces', {}).get(opp, []):
                    if not opp_piece.get('captured') and opp_piece.get('row') == nr and opp_piece.get('col') == nc:
                        occupant = opp
                        break
                if occupant:
                    break

            if is_corner:
                if piece.get('type') == 'good' and is_exit_for_player(player, nr, nc):
                    moves.append([pid, nr, nc])
            else:
                if occupant is None or occupant != player:
                    moves.append([pid, nr, nc])

    return moves


def estimate_capture_probabilities(state: Dict, player: str, move: List[int]) -> Optional[Tuple[float, float]]:
    """
    Estimate probability the capture target is good vs evil using remaining counts, exit proximity,
    and positional progress (movement-based belief proxy).
    Returns (prob_good, prob_evil) or None if no capture happens.
    """
    opponent = get_opponent_id(player)
    _, row, col = move

    target_piece = None
    for piece in state.get('pieces', {}).get(opponent, []):
        if not piece.get('captured') and piece.get('row') == row and piece.get('col') == col:
            target_piece = piece
            break

    if target_piece is None:
        return None

    prob_good, prob_evil = piece_type_probabilities(state, opponent, target_piece)

    revealed = state.get('revealed', {}).get(opponent, [])
    if target_piece['id'] in revealed:
        return (prob_good, prob_evil)

    progress = 1 - (min(distance_to_exit(opponent, row, col), 8) / 8)
    progress_bias = max(0.0, progress * 0.05)
    prob_good += progress_bias
    prob_evil -= progress_bias

    home_rows = [0, 1] if opponent == '1' else [4, 5]
    if row in home_rows:
        prob_evil += 0.04
        prob_good -= 0.04

    if prob_good < 0:
        prob_good = 0.0
    if prob_evil < 0:
        prob_evil = 0.0
    total = prob_good + prob_evil
    if total == 0:
        return (0.5, 0.5)
    return (prob_good / total, prob_evil / total)


def probabilistic_outcomes(state: Dict, player: str, move: List[int]) -> List[Tuple[float, Dict]]:
    """
    Generate probabilistic outcome states for a move that may capture an unrevealed piece.
    Uses belief-based probabilities instead of uniform assumptions.
    """
    opponent = get_opponent_id(player)
    _, row, col = move

    # Find the target piece if any
    target_piece = None
    for piece in state.get('pieces', {}).get(opponent, []):
        if not piece.get('captured') and piece.get('row') == row and piece.get('col') == col:
            target_piece = piece
            break

    if target_piece is None:
        return [(1.0, apply_move_to_state(state, player, move))]

    revealed = state.get('revealed', {}).get(opponent, [])
    if target_piece['id'] in revealed:
        return [(1.0, apply_move_to_state(state, player, move))]

    probs = estimate_capture_probabilities(state, player, move)
    if probs is None:
        return [(1.0, apply_move_to_state(state, player, move))]

    prob_good, prob_evil = probs
    outcomes: List[Tuple[float, Dict]] = []

    if prob_good > 0:
        good_state = apply_move_to_state(state, player, move, override_capture_type='good')
        outcomes.append((prob_good, good_state))
    if prob_evil > 0:
        evil_state = apply_move_to_state(state, player, move, override_capture_type='evil')
        outcomes.append((prob_evil, evil_state))
    return outcomes


MINIMAX_TABLE: Dict = {}
EXPECTI_TABLE: Dict = {}


def budget_exceeded(ctx: Optional[Dict]) -> bool:
    """Check time/node limits and increment node counter."""
    if ctx is None:
        return False
    ctx['node_count'] += 1
    if ctx.get('node_limit') is not None and ctx['node_count'] > ctx['node_limit']:
        return True
    if ctx.get('time_limit') is not None:
        if (time.time() - ctx['start']) > ctx['time_limit']:
            return True
    return False


def minimax(
    state: Dict,
    player_to_move: str,
    max_player: str,
    depth: int,
    alpha: float,
    beta: float,
    table: Optional[Dict] = None,
    ctx: Optional[Dict] = None
) -> float:
    """
    Alpha-beta minimax over the fully observed state (fallback when probabilistic search is off).
    """
    if table is None:
        table = MINIMAX_TABLE

    key = (player_to_move, max_player, depth, state_signature(state))
    if key in table:
        return table[key]

    if budget_exceeded(ctx):
        tmp_game = GhostsGame(json.dumps(state), 'playing', player_to_move, max_player)
        tmp_game._state = state
        return evaluate_state(tmp_game, state, perspective=max_player)

    winner = detect_winner(state)
    if winner is not None:
        val = 1e6 if winner == max_player else -1e6
        table[key] = val
        return val

    if depth == 0:
        tmp_game = GhostsGame(json.dumps(state), 'playing', player_to_move, max_player)
        tmp_game._state = state
        val = evaluate_state(tmp_game, state, perspective=max_player)
        table[key] = val
        return val

    moves = get_valid_moves_for_state(state, player_to_move)
    if not moves:
        val = -1e5 if player_to_move == max_player else 1e5
        table[key] = val
        return val

    maximizing = player_to_move == max_player
    best_val = float('-inf') if maximizing else float('inf')
    opponent = get_opponent_id(player_to_move)

    # Move ordering by heuristic
    scored_moves = []
    for move in moves:
        child_state = apply_move_to_state(state, player_to_move, move)
        tmp_game = GhostsGame(json.dumps(child_state), 'playing', opponent, max_player)
        tmp_game._state = child_state
        scored_moves.append((evaluate_state(tmp_game, child_state, perspective=max_player), move, child_state))
    scored_moves.sort(key=lambda x: x[0], reverse=maximizing)

    for _, move, child_state in scored_moves:
        val = minimax(child_state, opponent, max_player, depth - 1, alpha, beta, table, ctx)
        if maximizing:
            best_val = max(best_val, val)
            alpha = max(alpha, best_val)
        else:
            best_val = min(best_val, val)
            beta = min(beta, best_val)
        if beta <= alpha:
            break

    table[key] = best_val
    return best_val


def expectiminimax(
    state: Dict,
    player_to_move: str,
    max_player: str,
    depth: int,
    branch_limit: int,
    table: Optional[Dict] = None,
    ctx: Optional[Dict] = None
) -> float:
    """
    Expectiminimax with chance nodes for unrevealed captures and transposition caching.

    Args:
        state: Current game state.
        player_to_move: Whose turn it is in the simulated state.
        max_player: The player we are optimizing for.
        depth: Remaining depth (plies).
        branch_limit: Limit explored moves per node after ordering.
    """
    if table is None:
        table = EXPECTI_TABLE

    key = (player_to_move, max_player, depth, state_signature(state))
    if key in table:
        return table[key]

    if budget_exceeded(ctx):
        tmp_game = GhostsGame(json.dumps(state), 'playing', player_to_move, max_player)
        tmp_game._state = state
        return evaluate_state(tmp_game, state, perspective=max_player)

    winner = detect_winner(state)
    if winner is not None:
        val = 1e6 if winner == max_player else -1e6
        table[key] = val
        return val

    if depth == 0:
        tmp_game = GhostsGame(json.dumps(state), 'playing', player_to_move, max_player)
        tmp_game._state = state
        val = evaluate_state(tmp_game, state, perspective=max_player)
        table[key] = val
        return val

    moves = get_valid_moves_for_state(state, player_to_move)
    if not moves:
        val = -1e5 if player_to_move == max_player else 1e5
        table[key] = val
        return val

    opponent = get_opponent_id(player_to_move)
    maximizing = player_to_move == max_player

    # Order moves by heuristic to trim branching
    scored_moves = []
    for move in moves:
        child_state = apply_move_to_state(state, player_to_move, move)
        tmp_game = GhostsGame(json.dumps(child_state), 'playing', opponent, max_player)
        tmp_game._state = child_state
        scored_moves.append((evaluate_state(tmp_game, child_state, perspective=max_player), move))

    scored_moves.sort(key=lambda x: x[0], reverse=maximizing)
    if len(scored_moves) > branch_limit:
        scored_moves = scored_moves[:branch_limit]

    if maximizing:
        best = float('-inf')
        for _, move in scored_moves:
            outcomes = probabilistic_outcomes(state, player_to_move, move)
            exp_val = 0.0
            for prob, outcome_state in outcomes:
                exp_val += prob * expectiminimax(outcome_state, opponent, max_player, depth - 1, branch_limit, table, ctx)
            best = max(best, exp_val)
        table[key] = best
        return best
    else:
        best = float('inf')
        for _, move in scored_moves:
            outcomes = probabilistic_outcomes(state, player_to_move, move)
            exp_val = 0.0
            for prob, outcome_state in outcomes:
                exp_val += prob * expectiminimax(outcome_state, opponent, max_player, depth - 1, branch_limit, table, ctx)
            best = min(best, exp_val)
        table[key] = best
        return best


def move_captures_known_last_evil(state: Dict, player: str, move: List[int]) -> bool:
    opponent = get_opponent_id(player)
    _, row, col = move
    for piece in state.get('pieces', {}).get(opponent, []):
        if piece.get('captured'):
            continue
        if piece.get('row') == row and piece.get('col') == col:
            if piece.get('type') == 'evil':
                opp_evil = count_pieces_in_state(state, opponent, 'evil')
                opp_good = count_pieces_in_state(state, opponent, 'good')
                if opp_evil == 1 and opp_good > 0:
                    return True
            break
    return False


def find_exit_defense_move(state: Dict, player: str, moves: List[List[int]]) -> Optional[List[int]]:
    opponent = get_opponent_id(player)
    threats: List[Tuple[int, int]] = []
    for piece in state.get('pieces', {}).get(opponent, []):
        if piece.get('captured'):
            continue
        prob_good, _ = piece_type_probabilities(state, opponent, piece)
        if prob_good < 0.3:
            continue
        dist = distance_to_exit(opponent, piece.get('row'), piece.get('col'))
        if dist <= 1:
            threats.append((piece.get('row'), piece.get('col')))
    if not threats:
        return None
    threat_set = set(threats)
    defensive_moves = [m for m in moves if (m[1], m[2]) in threat_set]
    if defensive_moves:
        return random.choice(defensive_moves)
    return None


def select_move(
    game: GhostsGame,
    lookahead: int = 3,
    branch_limit: int = 12,
    use_expectiminimax: bool = True,
    time_limit: float = 1.5,
    node_limit: int = 8000
) -> Optional[List[int]]:
    """
    Choose a move using expectiminimax with probabilistic capture handling, risk adjustments,
    and iterative deepening within time/node budgets.

    Args:
        game: Active GhostsGame instance.
        lookahead: Max depth in plies.
        branch_limit: Max number of child moves to explore (after ordering).
        use_expectiminimax: If False, fall back to deterministic alpha-beta minimax.
        time_limit: Wall-clock seconds budget for the search.
        node_limit: Maximum evaluated nodes across the deepening loop.

    Returns:
        Move list [piece_id, row, col] or None if no moves exist.
    """
    moves = game.get_valid_moves()
    if not moves:
        return None

    max_player = game.my_player
    current_player = game.current_player
    base_state = game.state

    opponent = get_opponent_id(current_player)

    filtered_moves = [m for m in moves if not move_captures_known_last_evil(base_state, current_player, m)]
    if filtered_moves:
        moves = filtered_moves

    total_remaining = sum(
        1 for pieces in base_state.get('pieces', {}).values() for p in pieces if not p.get('captured')
    )

    dynamic_depth = lookahead
    if total_remaining <= 10:
        dynamic_depth = max(dynamic_depth, 5)
    elif total_remaining >= 14:
        dynamic_depth = min(dynamic_depth, 4)

    dynamic_branch = branch_limit
    if total_remaining <= 10:
        dynamic_branch = branch_limit + 3

    immediate_wins: List[List[int]] = []
    for move in moves:
        child_state = apply_move_to_state(base_state, current_player, move)
        if detect_winner(child_state) == max_player:
            immediate_wins.append(move)
    if immediate_wins:
        return random.choice(immediate_wins)

    defensive_move = find_exit_defense_move(base_state, current_player, moves)
    if defensive_move:
        return defensive_move

    ordered = []
    for move in moves:
        next_state = apply_move_to_state(base_state, current_player, move)
        score = evaluate_state(game, next_state, perspective=max_player)
        capture_probs = estimate_capture_probabilities(base_state, current_player, move)
        if capture_probs:
            prob_good, prob_evil = capture_probs
            remaining_evil = count_pieces_in_state(base_state, opponent, 'evil')
            if prob_evil >= 0.65 and remaining_evil <= 1:
                score -= 2e5
            if prob_good >= 0.6:
                score += 5e4
        ordered.append((score, move))

    ordered.sort(key=lambda x: x[0], reverse=True)
    if len(ordered) > dynamic_branch:
        ordered = ordered[:dynamic_branch]

    best_move = ordered[0][1]
    best_score = float('-inf')

    ctx = {
        'start': time.time(),
        'time_limit': time_limit,
        'node_limit': node_limit,
        'node_count': 0
    }

    for depth in range(1, dynamic_depth + 1):
        if budget_exceeded(ctx):
            break

        iter_best_score = float('-inf')
        iter_best_moves: List[List[int]] = []

        for _, move in ordered:
            if budget_exceeded(ctx):
                break

            risk_adjust = 0.0
            capture_probs = estimate_capture_probabilities(base_state, current_player, move)
            remaining_evil = count_pieces_in_state(base_state, opponent, 'evil')

            if capture_probs is not None:
                prob_good, prob_evil = capture_probs
                if prob_evil >= 0.65 and remaining_evil <= 1:
                    risk_adjust -= 4e5
                if prob_good >= 0.6:
                    risk_adjust += 6e4

            if use_expectiminimax:
                exp_val = 0.0
                outcomes = probabilistic_outcomes(base_state, current_player, move)
                for prob, next_state in outcomes:
                    exp_val += prob * expectiminimax(next_state, opponent, max_player, depth - 1, dynamic_branch, ctx=ctx)
                score = exp_val + risk_adjust
            else:
                next_state = apply_move_to_state(base_state, current_player, move)
                score = minimax(next_state, opponent, max_player, depth - 1, float('-inf'), float('inf'), ctx=ctx) + risk_adjust

            if score > iter_best_score:
                iter_best_score = score
                iter_best_moves = [move]
            elif abs(score - iter_best_score) < 1e-9:
                iter_best_moves.append(move)

        if iter_best_moves:
            best_move = random.choice(iter_best_moves)
            best_score = iter_best_score

    return best_move


def my_agent(game: GhostsGame) -> List:
    """
    Solver that picks a varied setup and uses probabilistic expectiminimax during play.

    Args:
        game: GhostsGame object with helper methods.

    Returns:
        Setup (list of 8 pieces) or Move (list of [piece_id, row, col]).
    """

    if game.is_setup_phase() and not game.setup_complete():
        return strategic_setup(game.my_player)

    return select_move(game, lookahead=5, use_expectiminimax=True, time_limit=1.7, node_limit=12000)


print("✅ Solver function defined")
print("   Remember to implement your algorithm before running!")

✅ Solver function defined
   Remember to implement your algorithm before running!


---
## Section 6: Test Your Solver (Optional)

Test parts of your implementation before playing a full game.

In [13]:
# Create a test state
base_test_state = {
    'pieces': {
        '1': [
            {'id': 0, 'row': 2, 'col': 1, 'type': 'good', 'captured': False},
            {'id': 1, 'row': 2, 'col': 3, 'type': 'good', 'captured': False},
            {'id': 2, 'row': 1, 'col': 1, 'type': 'good', 'captured': False},
            {'id': 3, 'row': 1, 'col': 3, 'type': 'good', 'captured': False},
            {'id': 4, 'row': 1, 'col': 2, 'type': 'evil', 'captured': False},
            {'id': 5, 'row': 1, 'col': 4, 'type': 'evil', 'captured': False},
            {'id': 6, 'row': 2, 'col': 2, 'type': 'evil', 'captured': False},
            {'id': 7, 'row': 2, 'col': 4, 'type': 'evil', 'captured': False},
        ],
        '2': [
            {'id': 0, 'row': 3, 'col': 1, 'type': 'good', 'captured': False},
            {'id': 1, 'row': 3, 'col': 4, 'type': 'good', 'captured': False},
            {'id': 2, 'row': 4, 'col': 1, 'type': 'good', 'captured': False},
            {'id': 3, 'row': 4, 'col': 4, 'type': 'good', 'captured': False},
            {'id': 4, 'row': 4, 'col': 2, 'type': 'evil', 'captured': False},
            {'id': 5, 'row': 4, 'col': 3, 'type': 'evil', 'captured': False},
            {'id': 6, 'row': 3, 'col': 2, 'type': 'evil', 'captured': False},
            {'id': 7, 'row': 3, 'col': 3, 'type': 'evil', 'captured': False},
        ]
    },
    'revealed': {'1': [], '2': []},
    'phase': 'playing'
}

# Scenario 2: good ghost near an exit; solver should try to finish
exit_race_state = {
    'pieces': {
        '1': [
            {'id': 0, 'row': 4, 'col': 0, 'type': 'good', 'captured': False},
            {'id': 1, 'row': 2, 'col': 2, 'type': 'evil', 'captured': False}
        ],
        '2': [
            {'id': 10, 'row': 5, 'col': 2, 'type': 'evil', 'captured': False},
            {'id': 11, 'row': 3, 'col': 3, 'type': 'good', 'captured': False}
        ]
    },
    'revealed': {'1': [], '2': []},
    'phase': 'playing'
}

# Scenario 3: capturing the last opponent evil ghost would cause a loss; solver should avoid it
avoid_last_evil_state = {
    'pieces': {
        '1': [
            {'id': 0, 'row': 2, 'col': 2, 'type': 'good', 'captured': False},
            {'id': 1, 'row': 0, 'col': 1, 'type': 'evil', 'captured': False}
        ],
        '2': [
            {'id': 10, 'row': 3, 'col': 2, 'type': 'evil', 'captured': False},  # last evil
            {'id': 11, 'row': 5, 'col': 5, 'type': 'good', 'captured': False}
        ]
    },
    'revealed': {'1': [], '2': []},
    'phase': 'playing'
}

def run_solver_on_state(state, current_player: str = '1', my_player: str = '1', description: str = ''):
    """Utility to print a board and run the solver once."""
    game = GhostsGame(json.dumps(state), 'playing', current_player, my_player)
    print(f"\n== {description or 'Scenario'} ==")
    game.print_board()
    move = my_agent(game)
    print(f"Chosen move: {move}")

# Baseline sample from the notebook
run_solver_on_state(base_test_state, description="Baseline sample state")

# Push a good ghost to the exit if possible
run_solver_on_state(exit_race_state, description="Good ghost near exit")

# Avoid capturing the final opponent evil ghost when alternatives exist
run_solver_on_state(avoid_last_evil_state, description="Avoid capturing last evil ghost")



== Baseline sample state ==

Player 1 (You) - Good: 4, Evil: 4
Player 2 (Opp) - Good: 4, Evil: 4
  0   1   2   3   4   5
┌───┬───┬───┬───┬───┬───┐
│ X │   │   │   │   │ X │ 0
├───┼───┼───┼───┼───┼───┤
│   │ G │ E │ G │ E │   │ 1
├───┼───┼───┼───┼───┼───┤
│   │ G │ E │ G │ E │   │ 2
├───┼───┼───┼───┼───┼───┤
│   │ ? │ ? │ ? │ ? │   │ 3
├───┼───┼───┼───┼───┼───┤
│   │ ? │ ? │ ? │ ? │   │ 4
├───┼───┼───┼───┼───┼───┤
│ X │   │   │   │   │ X │ 5
└───┴───┴───┴───┴───┴───┘

Legend: G=your good, E=your evil, ?=unknown opponent
        g=revealed good, e=revealed evil, X=exit

Chosen move: [7, 3, 4]

== Good ghost near exit ==

Player 1 (You) - Good: 1, Evil: 1
Player 2 (Opp) - Good: 1, Evil: 1
  0   1   2   3   4   5
┌───┬───┬───┬───┬───┬───┐
│ X │   │   │   │   │ X │ 0
├───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │ 1
├───┼───┼───┼───┼───┼───┤
│   │   │ E │   │   │   │ 2
├───┼───┼───┼───┼───┼───┤
│   │   │   │ ? │   │   │ 3
├───┼───┼───┼───┼───┼───┤
│ G │   │   │   │   │   │ 4
├───┼───┼─

---
## Section 7: Play the Game!

**Run this cell to test your solver against the AI**

In [14]:
STUDENT_TOKEN = 'NIKOLOZ+JULIETTE'  # e.g., 'JOHN-DOE'
SOLVER = my_agent  # Change to manual_player_solver to play manually
MULTIPLAYER = True
MATCH_ID = None
NUM_GAMES = 5

result = play_game(
    solver=SOLVER,
    base_url=BASE_URL,
    token=STUDENT_TOKEN,
    game_type='ghosts',
    game_class=GhostsGame,
    multiplayer=MULTIPLAYER,
    match_id=MATCH_ID,
    num_games=NUM_GAMES,
    debug=False,
    verbose=True
)

stats, all_results = result
print("\n📊 Summary:")
print(f"   Record: {stats['wins']}W - {stats['losses']}L - {stats['draws']}D")
print(f"   Win Rate: {stats['win_rate']*100:.1f}%")

🎮 Creating new match: 5 x ghosts
   Match ID: 2885
🔗 Joining match 2885...
   You are player: 1
⏳ Waiting for opponent to join...

🎮 GAME 1/5


🎯 SETUP PHASE - Player 1
   Submitting your ghost setup...
   ✅ Setup submitted!
⏳ Waiting for opponent to complete setup...
⏳ Waiting for opponent to complete setup...
⏳ Waiting for opponent to complete setup...

Player 1 (You) - Good: 4, Evil: 4
Player 2 (Opp) - Good: 4, Evil: 4
  0   1   2   3   4   5
┌───┬───┬───┬───┬───┬───┐
│ X │ G │ E │ E │ G │ X │ 0
├───┼───┼───┼───┼───┼───┤
│   │ E │ G │ G │ E │   │ 1
├───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │ 2
├───┼───┼───┼───┼───┼───┤
│   │   │   │   │   │   │ 3
├───┼───┼───┼───┼───┼───┤
│   │ ? │ ? │ ? │ ? │   │ 4
├───┼───┼───┼───┼───┼───┤
│ X │ ? │ ? │ ? │ ? │ X │ 5
└───┴───┴───┴───┴───┴───┘

Legend: G=your good, E=your evil, ?=unknown opponent
        g=revealed good, e=revealed evil, X=exit

⏳ Waiting for opponent (Player 2)...

Player 1 (You) - Good: 4, Evil: 4
Player 2 (Opp) - Good: 4